In [14]:
import pandas as pd
import numpy as np
import pyarrow
import pickle

# Models
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Question 1: Download the data
- Read the data, how many columns are there?

In [4]:
# Get current working directory
!pwd

/Users/samlafell/Desktop/mlops_zoomcamp_sam/01-intro


In [11]:
import pathlib
data_path = pathlib.Path('../data').resolve()

In [13]:
def import_parquet_data(data_path, file):
    return pd.read_parquet(pathlib.Path(data_path, file), engine = 'pyarrow')

df = import_parquet_data(data_path, 'yellow_tripdata_2021-01.parquet')

The data has 19 columns

## Question 2. Computing Duration
- What's the standard deviation fo trips duration in January?

In [4]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.50,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.00,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.50,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.00,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.50,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2463926,2,2022-01-31 23:36:53,2022-01-31 23:42:51,NaN,1.32,NaN,None,90,170,0,8.00,0.0,0.5,2.39,0.0,0.3,13.69,NaN,NaN
2463927,2,2022-01-31 23:44:22,2022-01-31 23:55:01,NaN,4.19,NaN,None,107,75,0,16.80,0.0,0.5,4.35,0.0,0.3,24.45,NaN,NaN
2463928,2,2022-01-31 23:39:00,2022-01-31 23:50:00,NaN,2.10,NaN,None,113,246,0,11.22,0.0,0.5,2.00,0.0,0.3,16.52,NaN,NaN
2463929,2,2022-01-31 23:36:42,2022-01-31 23:48:45,NaN,2.92,NaN,None,148,164,0,12.40,0.0,0.5,0.00,0.0,0.3,15.70,NaN,NaN


In [5]:
df['Duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df.Duration = df.Duration.apply(lambda td: td.total_seconds() / 60)
df['Duration'].std() # numpy standard deviation
# 46.45

46.44530513776802

## Question 3. Dropping Outliers
- Keep only between 1 and 60 minute rides (inclusive)

In [6]:
 # Use DataFrame.loc[] to filter by multiple conditions
df2 = df.loc[(df['Duration']>=1) & (df['Duration'] <= 60), :]

In [7]:
assert df2['Duration'].min() > .98, 'Check on Min Value Filtering'
assert df2['Duration'].max() < 60.2, 'Check on Max Value Filtering'

In [8]:
df2.shape[0] / df.shape[0] # 98.27%

0.9827547930522406

## Q4. One Hot Encoding
- Turn DF into list of dicts
- Fit DicVectorizor
- Get a feature matrix

- What's the dimensionality of the matrix?

In [9]:
# Only keep pickup and dropoff location columns
categorical = ['PULocationID', 'DOLocationID']
df2[categorical] = df2[categorical].astype(str)

/tmp/ipykernel_1907/222181778.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[categorical] = df2[categorical].astype(str)


In [26]:
# Turn DF into list of dicts
train_dicts = df2[categorical].to_dict(orient='records')

In [11]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [13]:
target = 'Duration'
y_train = df2[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

6.986191065500608

## Import Feb Data to Validate

In [15]:
df = import_parquet_data(data_path, 'yellow_tripdata_2022-02.parquet')

In [18]:
# Create Target
df['Duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df.Duration = df.Duration.apply(lambda td: td.total_seconds() / 60)

# Remove Outliers
df_feb = df.loc[(df['Duration']>=1) & (df['Duration'] <= 60), :]

In [19]:
# Only keep pickup and dropoff location columns
categorical = ['PULocationID', 'DOLocationID']
df_feb[categorical] = df_feb[categorical].astype(str)

/tmp/ipykernel_1907/2963711502.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feb[categorical] = df_feb[categorical].astype(str)


In [20]:
# Turn DF into list of dicts
valid_dicts = df_feb[categorical].to_dict(orient='records')

In [23]:
x_valid = dv.transform(valid_dicts)
y_valid = df_feb[target].values

In [25]:
y_pred = lr.predict(x_valid)
mean_squared_error(y_valid, y_pred, squared=False)

7.786408015215065